**Show all your work for full credit. Each source code you submit should include detailed comments and instructions on how to run it in order to confirm that it works as expected. If the program that does not run or throws runtime errors, it cannot be graded. You can refer to the programming guidelines from the TAs here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

**This is an individual assignment and each student should work on their own. Ensure you don't share any code online or with others (note, using Replit, GitHub and similar online platforms can make your code accessible to others).**

**To submit the assignment, you need to use Jupyter Notebook with the provided cell blocks and follow the naming conventions and instructions posted here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and section below:

<font color='red' size="4">Import any additional libraries you need in the same code block that you use it.</font>

In [ ]:
NAME = "Shruthilaya Arun"
#SECTION = "472"
SECTION = "672"

---

## Question 1c - Review the relevant code samples provided on Canvas. Note: SimonX/Y indicates block size X and key size Y. The same applies to SpeckX/Y. Then, using Python crypto libraries in a secure way, implement the following:

In [ ]:
#! pip install SimonSpeck
#! pip install pycryptodome

from Crypto.Cipher import AES
from Crypto.Hash import CMAC
from simon import SimonCipher
from Crypto.Random import get_random_bytes
from Crypto.Util.number import bytes_to_long , long_to_bytes
import hmac
import secrets

In [5]:
from Cryptodome.Cipher import AES
from Cryptodome.Hash import CMAC
from simon import SimonCipher
from Cryptodome.Random import get_random_bytes
from Cryptodome.Util.number import bytes_to_long , long_to_bytes
import hmac
import secrets

### Q1c [15 points] Implement EtM using Simon128/256-CTR as the cipher and AES-CMAC as the MAC. Your implementation should support multiple plaintext blocks, and include decryption & mac verification support.

In [6]:
# 5 points for this code block
from cryptography.hazmat.primitives import cmac
from cryptography.hazmat.primitives.ciphers import algorithms
from cryptography.exceptions import InvalidSignature

def cmac_aes(ctxt: bytes, key: bytes) -> bytes:
    """
    Generate the AES-CMAC tag from the ctxt,
    using the CMAC and AES libraries.
    Return tag as bytes.
    """
    tag = b"" 
    # YOUR CODE HERE
    cmac_obj=cmac.CMAC(algorithms.AES(key)) # cmac initialise
    cmac_obj.update(ctxt) # add ctxt
    tag=cmac_obj.finalize() # get tag
    return tag

def cmac_aes_verify(tag: bytes, ctxt: bytes, key: bytes) -> bytes:
    """
    Verify the AES-CMAC tag using the CMAC and AES libraries.
    Return True if the tag matches, else false
    Hint: Use try except block to handle errors
    """
    # YOUR CODE HERE
    cmac_obj=cmac.CMAC(algorithms.AES(key))
    cmac_obj.update(ctxt)
    try:
        cmac_obj.verify(tag) # verify tag
        return True
    except InvalidSignature:
        return False  


In [7]:
ptxt = b"this  a random message that is sufficiently long"
key = b'\x0f?\xb9c1\xb6\x91\xe8\x8c\xc2\xbf\x10\xab\x12b\x98\x82\xf3h\xaa\x19! \xb99\x0fi1\xdc3\xb1\xcf'
tag = cmac_aes(ptxt, key)
assert tag == b'\xb2%\xf5\x1cV\x94\xe1\xa3\xf3\x98"3\x08<\xe5\xe0'
verify = cmac_aes_verify(tag, ptxt, key)
assert verify == True

In [8]:
ptxt = b"this  a random message that is sufficiently long"
key = b'\x0f?\xb9c1\xb6\x91\xe8\x8c\xc2\xbf\x10\xab\x12b\x98\x82\xf3h\xaa\x19! \xb99\x0fi1\xdc3\xb1\xcf'
tag = b'\xb2%\xf5\x1cV\x94\xe1\xa3\xf3\x98"3\x08<\xe5\xe1'
verify = cmac_aes_verify(tag, ptxt, key)
assert verify == False

In [9]:
# 5 points for encrypt/decrypt
def encrypt_simon(ptxt: bytes, key: bytes) -> (bytes, bytes):
    """
    Encrypt the ptxt using Simon128/256-CTR. Use the given key. 
    Return the ctxt and generated nonce as bytes.
    Use long_to_bytes() and bytes_to_long() for bytes <-> int conversions.
    Use default counter argument for Simon Cipher.
    Simon Cipher should take nonce as an argument.
    """
    ctxt = b""
    nonce = b""
    # YOUR CODE HERE 
    nonce=secrets.token_bytes(16) # generate nonce
    cipher=SimonCipher(bytes_to_long(key), key_size=256, block_size=128, mode='CTR',init=bytes_to_long(nonce)) # simon 128/256 CTR
    block_size=16 # block 
    padded_ptxt=ptxt + b'\x00' * (block_size - len(ptxt) % block_size)  # add padding
    for i in range(0, len(padded_ptxt), block_size):
        block=padded_ptxt[i:i + block_size] # get block
        ctxt+=long_to_bytes(cipher.encrypt(bytes_to_long(block))) 
    return ctxt,nonce

    

def decrypt_simon(ctxt: bytes, nonce: bytes, key: bytes) -> (bytes):
    """
    Decrypt the ctxt using the Simon128/256-CTR. Use the given key and nonce. 
    Return the decrypted ptxt.
    Use long_to_bytes() and bytes_to_long() for bytes <-> int conversions.
    Use default counter argument for Simon Cipher.
    Simon Cipher should take nonce as an argument.
    """
    ptxt = b""
    # YOUR CODE HERE
    cipher=SimonCipher(bytes_to_long(key),key_size=256,block_size=128,mode='CTR',init=bytes_to_long(nonce)) 
    block_size=16
    for i in range(0,len(ctxt),block_size):
        block=ctxt[i:i+block_size]
        decrypted_block=long_to_bytes(cipher.decrypt(bytes_to_long(block)),block_size) # decrypt ptxt
        ptxt+=decrypted_block
    return ptxt.rstrip(b'x\00') # remove zeros


In [10]:
ptxt = b"this  a random message that is sufficiently long"
key = b'\x0f?\xb9c1\xb6\x91\xe8\x8c\xc2\xbf\x10\xab\x12b\x98\x82\xf3h\xaa\x19! \xb99\x0fi1\xdc3\xb1\xcf'
ctxt = b'w\x13\xb6\xb8q>!\xf5\x93dm5#\x88\x9d\x8doTAts\xbcN\xc8\xe0\x19\x01?]\x82\xfa\xd1_\xefm\xa2\x1c\xf1\xf1J\xad\xa2\x17\xde\xa7\xc8\x98\xbd' 
nonce = b'\x89\x0e\xbc\x9c\xb8\xc5Be\x1b\x9e\xd8\x86>\xfd\x97m\xf8\x0fB\xfd\x96\xf4$\x8a\x1f\x8d\xc8G\xda\xb9/\xd5\xf0)hLD\x89\xe7\x0c\xf8\x1cM)i+\xad\x05\xfd\xb7\xa5\xd0\xac\xe2\xd2\x00\xa9,\xec\xea5\xa7\xd9\x1a=G\xdan\\!\xff\xe3\xa6\xd2\x87\x04aQ\xe4\x0f\xf8\x96\xe2\xb8,\xf4\x99\x1c,\x93\xce\xc0\x91\xcc1V\x99\xde\xc0\xe4m\xed[\x04\xcb\x10\x12`\x99\x9c\x89Ay\x8d\x04\x13]\xbb\xa0\xf1\xe2\x0f\x941\xb0\x15\xe4\xa0'

ptxt_decrypt = decrypt_simon(ctxt, nonce, key)
assert ptxt_decrypt == ptxt

In [11]:
ptxt = b"this  a random message that is sufficiently long"
key = b'\x0f?\xb9c1\xb6\x91\xe8\x8c\xc2\xbf\x10\xab\x12b\x98\x82\xf3h\xaa\x19! \xb99\x0fi1\xdc3\xb1\xcf'

ctxt, nonce = encrypt_simon(ptxt, key)
ptxt_decrypt = decrypt_simon(ctxt, nonce, key)
assert ptxt_decrypt == ptxt

# Ensure we generate a new nonce
ctxt_new, nonce_new = encrypt_simon(ptxt, key)
assert nonce_new != nonce
assert ctxt_new != ctxt

In [12]:
# [3 points] Implement you own EtM_encrypt() function and implement an example to demonstrate its use.
# You can have a hardcoded message, but the remaining elements must be securely generated.
# Print the ciphertext and tag result in hex.

In [23]:
# YOUR CODE HERE
def EtM_encrypt(ptxt: bytes, enc_key: bytes,mac_key:bytes) -> (bytes, bytes, bytes):
    
    ctxt,nonce=encrypt_simon(ptxt,enc_key) # encrypt ptxt
    tag=cmac_aes(ctxt, mac_key) # tag of ctxt
    
    return ctxt, nonce, tag



In [24]:
ptxt = b"This is a secret message"
enc_key = secrets.token_bytes(32)  # encryption key
mac_key=secrets.token_bytes(32) # mac key
ctxt, nonce, tag = EtM_encrypt(ptxt, enc_key,mac_key)
print("Ciphertext:", ctxt.hex())
print("Tag:", tag.hex())

Ciphertext (hex): ad2dc82df4044eb855f7b8dd85a726dd165447022b20eb26f552917b1be55fb3
Nonce (hex): 6322c115dcbac8af73986647fb33beb6
CMAC Tag (hex): b73b59358c0762b8a790bb1dbf7575b5


In [25]:
# hidden test cases

In [26]:
# [2 points] Implement you own EtM_decrypt() function. 
# Demostrate its use by decrypting and verifying the EaM_encrypt result above. 
# Print the plaintext as a byte string and tag result.

In [27]:
# YOUR CODE HERE
def EtM_decrypt(ctxt: bytes, nonce: bytes, tag: bytes, enc_key: bytes,mac_key:bytes) -> bytes:
    if not cmac_aes_verify(tag, ctxt, mac_key):
            raise ValueError("Invalid MAC") # verify mac
    return decrypt_simon(ctxt, nonce, enc_key)

In [28]:
ptxt = b"This is a secret message"
enc_key = secrets.token_bytes(32)  # encryption key
mac_key=secrets.token_bytes(32) # mac key
ctxt, nonce, tag = EtM_encrypt(ptxt, enc_key,mac_key) # etm encrypt
try:
        decrypted_ptxt = EtM_decrypt(ctxt, nonce, tag, enc_key,mac_key) # etm decrypt
        print("Decrypted Plaintext:", decrypted_ptxt)
        print("MAC Tag Verified:", tag.hex())
except ValueError as e:
        print("Decryption failed:", str(e))

Decrypted Plaintext: b'This is a secret message'
MAC Tag Verified: 1279a1c91ff1c92626ef1c953b82eb4f


In [ ]:
# hidden test cases